In [9]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import cartopy
import numpy as np

from utils import get_data
from utils import tech_order, tech_colors
from utils import get_data_sftp

from datetime import datetime
from pytz import timezone
import os

plt.style.use("style.txt")
pd.options.display.float_format = '{:,.2f}'.format


Bad key legend.labelcolor in file style.txt, line 43 ('legend.labelcolor: black')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
def graph_pie_chart(self, df, bins=(0, 10, 30, 60, float("inf")), sizes=(100, 200, 300, 400), ax=None,
                        title="Power Capacity (GW)", legend=True):
        """
        Graphs the data from the dataframe to a map pie chart.
        The dataframe should have 3 columns, gen_load_zone, gen_type and value.
        """
        _, center_points = self._load_maps()

        if ax is None:
            ax = self.draw_base_map()
        df = df.merge(center_points, on="gen_load_zone")

        assert not df["gen_type"].isnull().values.any()
        colors = self._tools.get_colors()
        lz_values = df.groupby("gen_load_zone")[["value"]].sum()
        if (lz_values["value"] == 0).any():
            raise NotImplementedError("Can't plot when some load zones have total value of 0")
        lz_values["size"] = self._tools.pd.cut(lz_values.value, bins=bins, labels=sizes)
        if lz_values["size"].isnull().values.any():
            lz_values["size"] = 150
            warnings.warn("Not using variable pie chart size since values were out of bounds during cutting")
        for index, group in df.groupby("gen_load_zone"):
            x, y = group["geometry"].iloc[0].x, group["geometry"].iloc[0].y
            group_sum = group.groupby("gen_type")["value"].sum().sort_values()
            group_sum = group_sum[group_sum != 0].copy()

            tech_color = [colors[tech] for tech in group_sum.index.values]
            total_size = lz_values.loc[index]["size"]
            ratios = (group_sum / group_sum.sum()).values
            self._pie_plot(x, y, ratios, tech_color, total_size, ax)

        if legend:
            legend_points = []
            for size, label in zip(sizes, self._tools.create_bin_labels(bins)):
                legend_points.append(
                    ax.scatter([], [], c="k", alpha=0.5, s=size, label=str(label))
                )
            legend = ax.legend(
                handles=legend_points,
                title=title,
                labelspacing=0.75,
                bbox_to_anchor=(1, 0),
                framealpha=0,
                loc="lower left",
                fontsize="small",
                title_fontsize="small"
            )
            ax.add_artist(legend)  # Required, see : https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#multiple-legends-on-the-same-axes

            legend_points = []
            for tech in df["gen_type"].unique():
                legend_points.append(
                    ax.scatter([],[],c=colors[tech], marker="s", label=tech)
                )

            legend = ax.legend(
                handles=legend_points,
                ncol=5,
                loc="upper left",
                bbox_to_anchor=(0, 0),
                # framealpha=0,
                fontsize="small",
                title_fontsize="small",
                labelspacing=0.3
            )
            ax.add_artist(legend)

        return ax

In [ ]:
# dispatch = tools.get_dataframe("dispatch_zonal_annual_summary.csv").rename({"Energy_GWh_typical_yr": "value"},                                                                           axis=1)
# dispatch = tools.transform.gen_type(dispatch)
# dispatch = dispatch.groupby(["gen_type", "gen_load_zone"], as_index=False)["value"].sum()
# dispatch["value"] *= 1e-3
# tools.maps.graph_pie_chart(dispatch, bins=(0, 10, 100, 200, float("inf")), title="Yearly Dispatch (TWh)")

In [6]:
# scenario = ['s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14',
#                  's15','s16','s17','s18','s19','s20','s21','s22','s23','s24','s25']
scenario = ['s1','s5','s21','s25']
short_names={'s1':'1','s2':'2','s3':'3','s4':'4','s5':'5','s6':'6','s7':'7','s8':'8',
             's9':'9','s10':'10','s11':'11','s12':'12','s13':'13','s14':'14','s15':'15',
             's16':'16','s17':'17','s18':'18','s19':'19','s20':'20','s21':'21','s22':'22',
             's23':'23','s24':'24','s25':'25'}

order={'1':0, '2':1,'3':2,'4':3,'5':4,'6':5,'7':6,'8':7,'9':8,'10':9,'11':10,'12':11,
       '13':12,'14':13,'15':14,'16':15,'17':16,'18':17,'19':18,'20':19,'21':20,'22':21,
       '23':22,'24':23,'25':24}
folder_to_save_results='results_test/'

#Check if the directory exists. If not, then create the directory.
if not os.path.exists(folder_to_save_results):
    os.makedirs(folder_to_save_results)

In [ ]:
#zones_under_analysis
# analysis_zones=['WA_SEATAC','WA_W','OR_W','CA_PGE_N','CA_PGE_BAY','CA_PGE_CEN',
#                 'CA_PGE_S','CA_SCE_CEN','CA_LADWP','CA_SCE_S','CA_SDGE','MEX_BAJA'] #these are coastal load zones
analysis_period=[2050]

In [7]:
# Load the shape file
wecc_load_areas = gpd.read_file('shape_files/wecc_load_areas.shp')

In [10]:
# Load the csv file from specified scenarios
fname = "dispatch.csv"

dispatch = get_data(scenario, fname)
dispatch  = dispatch .replace({"scenario": short_names})

FileNotFoundError: [Errno 2] No such file or directory: '/data/switch/wave_cases_v2/s1/outputs/dispatch.csv'

In [ ]:
# Load the shape file
wecc_load_areas = gpd.read_file('shape_files/wecc_load_areas.shp')

# Load the csv file
dispatch = pd.read_csv('dispatch.csv')

# Merge the shape file with the csv file based on the load zone
wecc_load_areas_dispatch = wecc_load_areas.merge(dispatch, on='load_zone')

# Group the data by load zone and calculate the sum of offshore wind and wave energy
grouped_dispatch = wecc_load_areas_dispatch.groupby('load_zone').sum()[['offshore_wind_energy', 'wave_energy']]

# Plot a pie chart for each load zone
for i, row in wecc_load_areas_dispatch.iterrows():
    # Calculate the size of the pie chart based on the total amount of offshore wind and wave energy dispatched
    size = (row['offshore_wind_energy'] + row['wave_energy']) / grouped_dispatch.loc[row['load_zone']].sum()
    # Create the pie chart
    plt.pie([row['offshore_wind_energy'], row['wave_energy']], radius=size, labels=['Offshore Wind Energy', 'Wave Energy'], autopct='%1.1f%%')
    # Set the title of the pie chart to the load zone
    plt.title(row['load_zone'])
    # Save the pie chart as an image file
    plt.savefig(f'{row["load_zone"]}.png')
    # Close the figure
    plt.close()
